# ATHENA Tutorial on MIBI Data

In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import seaborn as sns
import spatialHeterogeneity as sh
from spatialOmics import SpatialOmics
from tqdm import tqdm

root = Path()

sel1 = ['14', '33', '20', '29']
sel2 = ['5', '35', '4', '28']
spls = sel1 + sel2

attr = 'group_id'

## Data
Download the pre-processed data set. I combined the data from https://www.angelolab.com/mibi-data and https://mibi-share.ionpath.com/tracker/imageset (see [load-mibi-data.py]('./load-mibi-data.py'))

In [ ]:
so = sh.dataset.mibi()
# so = sh.dataset.mibi_pop()  # load populated MIBI dataset

In [ ]:
## Visualisation
ATHENA 

In [ ]:
# %%
sh.pl.napari_viewer(so, '1', ['Ta', 'Si', 'H3K25me3', 'dsDNA', 'CD45', 'Background'], attrs_key='channel',
                    index_key='stack_index')

# to load all channels
# sh.pl.napari_viewer(so, '1', so.var['1'].channel, attrs_key='channel', index_key='stack_index')

In [ ]:
# %%
for spl in spls:
    if spl in so.masks:
        sh.pp.extract_centroids(so, spl)

fig, axs = plt.subplots(1, 2, figsize=(9, 4))
sh.pl.spatial(so, spl, None, ax=axs[0])
sh.pl.spatial(so, spl, 'dsDNA', ax=axs[1])
fig.show()

In [ ]:
# %% explore radius
from scipy.spatial.distance import cdist
from spatialHeterogeneity.graph_builder.constants import GRAPH_BUILDER_DEFAULT_PARAMS

# get an impression of the number of neighbors for different radii
df = so.obs[spl][['x', 'y']]
radius = 50
cell2cell_dists = cdist(df, df)
g = plt.hist((cell2cell_dists <= radius).sum(1))
plt.show()

config_radius = GRAPH_BUILDER_DEFAULT_PARAMS['radius']
config_radius['builder_params']['radius'] = radius

In [ ]:
# %% compute graphs
for spl in spls:
    if spl in so.masks:
        try:
            sh.graph.build_graph(so, spl, 'radius', config=config_radius)
            sh.graph.build_graph(so, spl, 'contact')
        except Exception as e:
            print(f'{spl} had exception:\n{e}')

fig, axs = plt.subplots(figsize=(16, 16))
sh.pl.spatial(so, spl, 'SampleID', edges=True, graph_key='radius', ax=axs, cbar=False)
fig.show()

In [ ]:
# %% compute metrics at a sample level
for spl in spls:
    sh.metrics.richness(so, spl, attr, local=False, graph_key='contact')
    sh.metrics.shannon(so, spl, attr, local=False, graph_key='contact')
    sh.metrics.quadratic_entropy(so, spl, attr, local=False, graph_key='contact')

# estimated values are saved in so.spl
so.spl[[f'richness_{attr}', f'shannon_{attr}']]

In [ ]:
# %%
fig, axs = plt.subplots(1, 2, figsize=(10, 4), dpi=100)
sh.pl.spatial(so, spl, attr, mode='mask', ax=axs.flat[0])
# sh.pl.spatial(so, sel1[0], attr, mode='mask', ax=axs.flat[0])
# sh.pl.spatial(so, sel2[0], attr, mode='mask', ax=axs.flat[1])
fig.show()

In [ ]:
# %% compute metrics at a cell level for all samples - this will take some time
for spl in tqdm(spls):
    sh.metrics.richness(so, spl, attr, local=True, graph_key='contact')
    sh.metrics.shannon(so, spl, attr, local=True, graph_key='contact')
    sh.metrics.quadratic_entropy(so, spl, attr, local=True, graph_key='contact', metric='cosine')

# estimated values are saved in so.obs
so.obs[spl].columns
so.obs[spl][['richness_group_id_contact', 'shannon_group_id_contact', 'quadratic_group_id_contact']].head()

In [ ]:
# %% visualize cell-level scores
fig, axs = plt.subplots(1, 4, figsize=(25, 12), dpi=300)
axs = axs.flat
sh.pl.spatial(so, spl, attr, mode='mask', ax=axs[0])
sh.pl.spatial(so, spl, f'richness_{attr}_contact', mode='mask', ax=axs[1], cbar_title=False, background_color='black')
sh.pl.spatial(so, spl, f'shannon_{attr}_contact', mode='mask', ax=axs[2], cbar_title=False, background_color='black')
sh.pl.spatial(so, spl, f'quadratic_{attr}_contact', mode='mask', ax=axs[3], cbar_title=False, background_color='black')
fig.show()

In [ ]:
# %%
fig, ax = plt.subplots(figsize=(6, 4))
sns.histplot(so.obs[spl][f'quadratic_{attr}_contact'], stat='probability', ax=ax)
ax.set_title(
    f"Quadratic entropy, PatientID {spl}\n Cell-level median: {so.obs[spl][f'quadratic_{attr}_contact'].median():.3f}, 'Sample-level: {so.spl.loc[spl][f'quadratic_{attr}']:.3f}")
fig.show()



In [ ]:
# %% immune infiltration

for spl in tqdm(spls):
    sh.neigh.infiltration(so, spl, 'cell_type', graph_key='contact')

so.spl.head()

In [ ]:
# %%
sh.neigh.infiltration(so, spl, 'cell_type', graph_key='radius', local=True)

# %%
fig, axs = plt.subplots(1, 2, figsize=(16, 8))
# here we use the numeric representation of cell_type to be able to plot the cell outline
sh.pl.spatial(so, spl, 'cell_type_id', mode='mask', ax=axs[0])
sh.pl.infiltration(so, spl, step_size=25, ax=axs[1])
fig.show()

In [ ]:
# %%
fsave = Path('mibi-final.h5py')
so.to_h5py(fsave)